In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
! cp '/content/drive/MyDrive/cs473/lazydata-20221104T020024Z-001.zip' '/content'
from zipfile import ZipFile
zip = ZipFile('/content/lazydata-20221104T020024Z-001.zip')
zip.extractall()

Mounted at /content/drive/


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import cv2
import pickle as pkl
import warnings
warnings.filterwarnings("ignore")
! pip install wandb
# ! pip install ray
! pip install pytorch_lightning
import random
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image, ImageOps
from torch.utils.data import Dataset, DataLoader, IterableDataset
import torchvision.transforms as transforms
import torchvision.transforms as T
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from torch.nn.functional import mse_loss
from torchvision import transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import torchvision.models as models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 32.6 MB/s 
     |████████████████████████████████| 168 kB 90.7 MB/s 
     |████████████████████████████████| 182 kB 94.3 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 168 kB 87.4 MB/s 
     |████████████████████████████████| 166 kB 111.6 MB/s 
     |████████████████████████████████| 166 kB 112.1 MB/s 
     |████████████████████████████████| 162 kB 114.3 MB/s 
     |████████████████████████████████| 162 kB 84.2 MB/s 
     |████████████████████████████████| 158 kB 89.6 MB/s 
     |████████████████████████████████| 157 kB 95.3 MB/s 
     |████████████████████████████████| 157 kB 109.7 MB/s 
     |████████████████████████████████| 157 kB 88.4 MB/s 
     |████████████████████████████████| 157 kB 114.7 MB/s 
     |████████████████████████████████| 157 kB 123.6 MB/s 
     |█████████████████████

In [ ]:
def get_mean_std(gray= False):
  mean_std = {'mean':[], 'std':[]}
  for view in range(3):
    if gray:
      all_out = transforms.ToTensor()(ImageOps.grayscale(Image.open(f'/content/lazydata/train/X/0/rgb/{view}.png')))
      for i in range(1, 3396):
        out = transforms.ToTensor()(ImageOps.grayscale(Image.open(f'/content/lazydata/train/X/{i}/rgb/{view}.png')))
        all_out += out
    else:
      all_out = transforms.ToTensor()(Image.open(f'/content/lazydata/train/X/0/rgb/{view}.png'))
      for i in range(1, 3396):
        out = transforms.ToTensor()(Image.open(f'/content/lazydata/train/X/{i}/rgb/{view}.png'))
        all_out += out
    mean_std['mean'].append(torch.mean(all_out/3396)) 
    mean_std['std'].append(torch.std(all_out/3396))
  return mean_std
mean_std = get_mean_std(gray= True)


In [ ]:
# Lazy loading
class LazyLoadDataset(Dataset):
    def __init__(self, path, mean_std = mean_std, train = None, transform = None, data_idx = None):
      self.mean_std = mean_std
      self.transform = transform
      path = path + ('train/' if train else 'test/')
      self.pathX = path + 'X/'
      self.pathY = path + 'Y/'
      self.train = train
      if data_idx is None:
          self.data = os.listdir(self.pathX)
      else:
          self.data = data_idx
        
    def __getitem__(self, idx):

        f = self.data[idx]

        # X
        # read rgb images
        path0 = self.pathX + f + '/rgb/0.png'
        path1 = self.pathX + f + '/rgb/1.png'
        path2 = self.pathX + f + '/rgb/2.png'

        img0, img1, img2 = Image.open(path0), Image.open(path1), Image.open(path2)

        # data augmentation
        if self.transform is not None:
          img0 = self.transform(img0)
          img1 = self.transform(img1)
          img2 = self.transform(img2)


        # normalization of RGB
        img0 = transforms.ToTensor()(img0)
        img1 = transforms.ToTensor()(img1) 
        img2 = transforms.ToTensor()(img2) 

        mean0, mean1, mean2 = self.mean_std['mean'][0], self.mean_std['mean'][1], self.mean_std['mean'][2]
        std0, std1, std2 = self.mean_std['std'][0], self.mean_std['std'][1], self.mean_std['std'][2]

        
        img0 = transforms.Normalize(mean0, std0)(img0)
        img1 = transforms.Normalize(mean1, std1)(img1)
        img2 = transforms.Normalize(mean2, std2)(img2)


        # read depth images
        depth = np.load(self.pathX + f + '/depth.npy')/1000
        depth = np.clip(depth, 0.0, 2.0)
        depth0, depth1, depth2 = depth[0, :, :], depth[1, :, :], depth[2, :, :]
        # normalization of depth


        """depth0 = (depth0 - depth0.mean())/ depth0.std()
        depth1 = (depth1 - depth1.mean())/ depth1.std()
        depth2 = (depth2 - depth2.mean())/ depth2.std()"""
        

        # concat info from three views
        input1 = np.concatenate((img0, depth0[None, :, :]),axis = 0)[None, :, :, :]
        input2 = np.concatenate((img1, depth1[None, :, :]),axis = 0)[None, :, :, :]
        input3 = np.concatenate((img2, depth2[None, :, :]),axis = 0)[None, :, :, :]
        combined = torch.from_numpy(np.concatenate((input1, input2, input3), axis = 0))
        # combined dimension: batch * 3 * 4 * 224 * 224 

        # Y
        if self.train:
            Y = torch.from_numpy(np.load(self.pathY + f + '.npy') * 1000)
            return combined, Y

        else:
          field_id = pkl.load(open(self.pathX + f + '/field_id.pkl', 'rb'))
          return combined, field_id

    
    def __len__(self):
        return len(self.data)

In [ ]:
def gray(img):
  return ImageOps.grayscale(img)

def train_val_split(split_ratio = 0.1):
    val_idx = random.sample(range(3396), math.floor(split_ratio * 3396))
    train_idx = list(set(range(3396)).difference(set(val_idx)))
    val_idx, train_idx = [str(i) for i in val_idx], [str(i) for i in train_idx]
    return train_idx, val_idx

train_idx, val_idx = train_val_split()
train_dataset = LazyLoadDataset('/content/lazydata/',mean_std = mean_std, train = True, data_idx = train_idx, transform = gray)
val_dataset = LazyLoadDataset('/content/lazydata/', mean_std = mean_std, train = True, data_idx = val_idx, transform = gray)
print(f'train:{len(train_dataset)}, val:{len(val_dataset)}')

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

"""for i in train_dataloader:
    print(i[0].shape)"""

train:3057, val:339


'for i in train_dataloader:\n    print(i[0].shape)'

# get the mean of the training set and set it to the bias of the last linear layer

In [ ]:
"""all_y = [i[1] for i in train_dataloader]
all_y = torch.concat((torch.concat(all_y[:-1], axis = 0), all_y[-1]), axis = 0)
y_mean = torch.mean(all_y, axis = 0).to(device)
y_mean"""
# tensor([ 47.0646,  52.7343, 118.5616,  70.6122,  -0.6648,  95.8811,  59.8242,
        #-45.2541,  49.1242,  56.6229,  35.7834, -37.9923])

KeyboardInterrupt: ignored

**Architecture: use Convnet layers to extract features of different views, concatenate extracted features and pass forward to linear layers.**

In [ ]:
import torchvision.models as models
class MyModel(pl.LightningModule):  
  def __init__(self, dropout2=0.2, lr = 1e-3, gray = False, pretrained= True):

    super().__init__()
    self.lr = lr
    self.pretrained = pretrained
    if gray:
      self.first_channel = 2
    else:
      self.first_channel = 4
  
    self.res = models.resnet50(pretrained=self.pretrained)

    self.res = self.res.float()
    self.res.conv1 = nn.Conv2d(self.first_channel, 64, kernel_size=7, stride=2, padding=3)
    self.res.fc = nn.Sequential(
          nn.Flatten(),
          nn.Dropout(dropout2),
          nn.Linear(self.res.fc.in_features, 64), 
          nn.Linear(64, 12),
          nn.ReLU(),
 
    )
    self.finallinear = nn.Linear(12*3, 12, bias = False)

        

  def forward(self, x):
      concat_in = torch.cat([x[:,0,:,:,:], x[:,1,:,:,:], x[:,2,:,:,:]])
  
      concat_out = self.res(concat_in)

      chunk_size = concat_out.shape[0]//3
      fea1, fea2, fea3 = torch.split(concat_out, chunk_size)
      features = torch.cat((fea1.view(-1, 12), fea2.view(-1, 12), fea3.view(-1, 12)), axis = 1)
      y_mean = torch.tensor([ 47.0646,  52.7343, 118.5616,  70.6122,  -0.6648,  95.8811,  59.8242,
    -45.2541,  49.1242,  56.6229,  35.7834, -37.9923], requires_grad=False).float().to(device)
      out = self.finallinear(features)+ y_mean
  
      return out+ y_mean

  def training_step(self, batch, batch_idx):
      x, y = batch[0], batch[1]
      self = self.float()
      out = self(x.float())
      loss = torch.sqrt(mse_loss(out, y.float()))
      self.log("train_loss", loss)
      return loss
    
  def validation_step(self, batch, batch_idx):
      x, y = batch[0], batch[1]
      self = self.float()
      out = self(x.float())
      loss = torch.sqrt(mse_loss(out, y.float()))
      self.log('val_loss', loss)

      return loss
  
  def validation_epoch_end(self, outputs):
      #print(f'outputs:{outputs}')
      avg_loss = torch.stack([x for x in outputs]).mean()
      self.log('val_loss', avg_loss)
      return avg_loss
  
  def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.parameters(), lr = self.lr)
      scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
      return optimizer
  
  def test_pred(self, batch, batch_idx):
      x = batch[0]
      out = self(x.float())
      return out
      
    



In [ ]:
import wandb
from pytorch_lightning.loggers import WandbLogger
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray import tune
#wandb.login()
os.makedirs('/content/output', exist_ok=True)
# 2301426a232e1ea72e6398e9d453f510f7dc48b5

In [ ]:
from pytorch_lightning.loggers import WandbLogger
model = MyModel(gray = True).to(device)
"""
wandb_logger = WandbLogger(name="Run_chk_gradients_metrics_topo", 
                            project="MyCNN", 
                            log_model="all")
wandb_logger.watch(model, log="all",log_freq=32)
"""

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath='/content/output',       # Don not specify dirpath if using WandB to k
    #dirpath = "checkpoints",
    filename='models-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min')


metrics = {"loss": "val_loss"}
tune_callbacks = TuneReportCallback(metrics, on="validation_end")

trainer = pl.Trainer(
    #logger = wandb_logger,
    callbacks = [checkpoint_callback],
    check_val_every_n_epoch = 1,
    max_epochs = 30,
    log_every_n_steps=2,  
    accelerator='auto',
    devices= 'auto')


history = trainer.fit(model=model, train_dataloaders = train_dataloader, val_dataloaders = val_dataloader)
#trainer.validate(model, val_dataloader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type   | Params
---------------------------------------
0 | res         | ResNet | 23.6 M
1 | finallinear | Linear | 432   
---------------------------------------
23.6 M    Trainable params
0         Non-trainable params
23.6 M    Total params
94.549    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


In [ ]:
# Resume training from checkpoint
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainer = pl.Trainer(
    resume_from_checkpoint = "/content/output/models-epoch=31-val_loss=3.36.ckpt", 
    callbacks = [checkpoint_callback],
    check_val_every_n_epoch = 1,
    max_epochs = 42,
    log_every_n_steps=2,  
    accelerator='auto',
    devices= 'auto')
history2 = trainer.fit(model = model,  train_dataloaders = train_dataloader, val_dataloaders = val_dataloader)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:55: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1908: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpo

Sanity Checking: 0it [00:00, ?it/s]

Training: 192it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=42` reached.


In [ ]:
import pandas as pd
device = 'cpu'
def inference(best_model_path):
  pretrained_model = MyModel.load_from_checkpoint(best_model_path, gray = True)
  pretrained_model.eval()
  pretrained_model.freeze()

  test_dataset = LazyLoadDataset('/content/lazydata/',mean_std = mean_std, train = False, transform = gray)
  test_dataloader = DataLoader(test_dataset, batch_size= 128, shuffle = False)

  all_output = None
  for i, batch in enumerate(test_dataloader):
    if i == 0:
      all_output = pretrained_model(batch[0])/1000
      all_ids = batch[1]
    else:
      output = pretrained_model(batch[0]) /1000
      all_output = tf.concat([all_output, output], axis=0)
      all_ids = tf.concat([all_ids, batch[1]], axis = 0)
  return all_output, all_ids

best_model_path = "/content/output/models-epoch=36-val_loss=3.31.ckpt"
out_path = '/content/gy654_submission.csv'
predictions, ids = inference(best_model_path)
titles = [f'FINGER_POS_{i}' for i in range(1, 13)]
submit_df = pd.DataFrame(predictions, columns = titles).astype("float")
submit_df['ID'] = np.array(ids).astype(int)
submit_df = submit_df[['ID'] + titles]
submit_df.to_csv(out_path, index = False)

In [ ]:
submit_df

,ID,FINGER_POS_1,FINGER_POS_2,FINGER_POS_3,FINGER_POS_4,FINGER_POS_5,FINGER_POS_6,FINGER_POS_7,FINGER_POS_8,FINGER_POS_9,FINGER_POS_10,FINGER_POS_11,FINGER_POS_12
0,3010,0.014586,0.058914,0.137564,0.022740,-0.000746,0.137596,0.038025,-0.061890,0.122363,0.057141,0.079431,-0.019595
1,3214,0.050537,0.054417,0.121935,0.047177,-0.002031,0.125992,0.032776,-0.054810,0.129691,0.054155,0.026112,-0.066221
2,1900,0.062926,0.053077,0.115488,0.050498,-0.000776,0.120531,0.040360,-0.054309,0.119817,0.054569,-0.010600,-0.070187
3,3812,0.062533,0.051077,0.101374,0.069456,-0.003799,0.085223,0.060173,-0.048497,0.066460,0.051740,0.031223,-0.058859
4,2502,0.067300,0.051003,0.102919,0.075858,-0.003262,0.098518,0.062641,-0.051498,0.106074,0.084941,0.035514,-0.036265
...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,3832,0.057285,0.052447,0.110610,0.075283,-0.005147,0.078227,0.065151,-0.047524,0.074787,0.049048,0.025311,-0.052182
845,1074,0.058057,0.052568,0.117286,0.107017,0.004494,0.059812,0.068463,-0.040762,0.014476,0.057703,0.027394,-0.045366
846,2367,0.027548,0.055698,0.130306,0.048811,-0.002754,0.123172,0.059532,-0.037896,0.022295,0.055754,-0.004473,-0.066093
847,1234,0.018631,0.054356,0.126578,0.076657,-0.005043,0.102025,0.081088,-0.034941,-0.001880,0.052942,0.063629,0.003912


# Hyperparameter search

In [ ]:
def train_robothand(config, num_epochs=10):
    model = MyModel()
    wandb_logger = WandbLogger(name="Run_chk_gradients_metrics_topo",         # WandB Run name
                               project="MyCNN",           # WandB project name
                               log_model=True             # send modelcheckpoint to WandB at end of training 
                              )
    wandb_logger.watch(model, log="all",log_freq=32)


    early_stopping = pl.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", 
            patience=2,
            mode="min")

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        monitor='val_loss',
        every_n_train_steps = 10,
        #dirpath='./mypl',       # Don not specify dirpath if using WandB to k
        dirpath = "checkpoints",
        filename='models-{epoch:02d}-{val_loss:.2f}',
        save_top_k=1,
        mode='min')


    metrics = {"loss": "val_loss"}
    tune_callbacks = TuneReportCallback(metrics, on="validation_end")

    trainer = pl.Trainer(
        logger=wandb_logger,
        callbacks=[checkpoint_callback],
        max_epochs = num_epochs,
        accelerator='auto',
        devices='auto')

    trainer.fit(model=model, train_dataloaders = train_dataloader)
    trainer.validate(model, val_dataloader)

#train_robothand(config, num_epochs=10)

config = {
    "lr": tune.loguniform(1e-4, 1e-3, 1e-2),
    "batch_size": tune.choice([32, 64]),
}


analysis = tune.run(
 tune.with_parameters(train_robothand, num_epochs=10),
 config=config,
 num_samples=3)

# snippet to save lazy data

In [ ]:
import os
import numpy as np
import pickle as pkl
import torch
from torchvision.utils import save_image

trainx = torch.load('/kaggle/input/ml-final/train/train/trainX.pt')
trainy = torch.load('/kaggle/input/ml-final/train/train/trainY.pt')
testx = torch.load('/kaggle/input/ml-final/test/test/testX.pt')

num_train = trainx[0].shape[0]
num_test = testx[0].shape[0]

os.makedirs('./lazydata', exist_ok=True)

# Save train data
os.makedirs('./lazydata/train', exist_ok=True)
os.makedirs('./lazydata/train/X', exist_ok=True)
os.makedirs('./lazydata/train/Y', exist_ok=True)
for i in range(num_train):
    os.makedirs('./lazydata/train/X/{}'.format(i), exist_ok=True)
    # rgb
    os.makedirs('./lazydata/train/X/{}/rgb'.format(i), exist_ok=True)
    for j in range(3):
        save_image(trainx[0][i][j]/255, './lazydata/train/X/{}/rgb/{}.png'.format(i, j))
    # depth
    depth = trainx[1][i].numpy()
    np.save('./lazydata/train/X/{}/depth.npy'.format(i), depth)
    # field id
    pkl.dump(trainx[2][i], open('./lazydata/train/X/{}/field_id.pkl'.format(i), 'wb'))

    y = trainy[0][i].numpy()
    np.save('./lazydata/train/Y/{}.npy'.format(i), y)
print("Saved train data")

# Save test data
os.makedirs('./lazydata/test', exist_ok=True)
os.makedirs('./lazydata/test/X', exist_ok=True)
for i in range(num_test):
    os.makedirs('./lazydata/test/X/{}'.format(i), exist_ok=True)
    # rgb
    os.makedirs('./lazydata/test/X/{}/rgb'.format(i), exist_ok=True)
    for j in range(3):
        save_image(testx[0][i][j]/255, './lazydata/test/X/{}/rgb/{}.png'.format(i, j))
    # depth
    depth = testx[1][i].numpy()
    np.save('./lazydata/test/X/{}/depth.npy'.format(i), depth)
    # field id
    pkl.dump(testx[2][i], open('./lazydata/test/X/{}/field_id.pkl'.format(i), 'wb'))

print("Saved test data")